In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import os
import random
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
a = "label_train_test.pkl"
with open(a, 'rb') as f:
    label, train, test = pickle.load(f)
print(len(train), len(test))

2252 551


In [4]:
data_path = './gnet2'
old_data = train + test
data = []
all_labels = set()
for file_ in old_data:
    basename = os.path.basename(file_)
    name = os.path.splitext(basename)[0]
    data.append(os.path.join(data_path, basename))
    label_ = label[name]
    ls = label_.split('+')
    for l in ls:
        all_labels.add(l)
    # print(name, label_)
print(len(all_labels), all_labels)

14 {'SCC', 'EC', 'CC', 'ASCH', 'VIRUS', 'ASCUS', 'HSIL', 'HSV', 'AGC', 'FUNGI', 'ACTINO', 'LSIL', 'TRI', 'NILM'}


In [24]:
df = pd.read_csv('./gnet2/TC17038155.csv')
df.head(10)

,mpp,area,patches,patch_i,patch_label,patch_probability,detect_label,detect_probability,classify_label,classify_probability,x,y,w,h
0,NaN,NaN,NaN,2098,HSIL-SCC_G,0.999281,HSIL-SCC_G,0.138170,HSIL_B,0.999990,47632,14706,344,356
1,NaN,NaN,NaN,199,HSIL-SCC_G,0.999966,HSIL-SCC_G,0.154858,HSIL_B,0.999954,9494,27512,370,248
2,NaN,NaN,NaN,1833,HSIL-SCC_G,0.865506,FUNGI,0.930550,FUNGI,0.999952,41208,32406,200,526
3,NaN,NaN,NaN,200,HSIL-SCC_G,0.998435,HSIL-SCC_G,0.625874,HSIL_B,0.999947,9474,27526,382,260
4,NaN,NaN,NaN,684,NORMAL,0.989545,PH,0.779710,MC,0.999944,19738,24840,210,184
5,NaN,NaN,NaN,1525,HSIL-SCC_G,0.576868,FUNGI,0.798580,FUNGI,0.999928,34846,44542,118,60
6,NaN,NaN,NaN,669,HSIL-SCC_G,0.993016,HSIL-SCC_G,0.830921,HSIL_B,0.999859,19398,11894,174,150
7,NaN,NaN,NaN,1462,NORMAL,0.435115,FUNGI,0.137338,FUNGI,0.999779,33408,32558,354,214
8,NaN,NaN,NaN,1880,HSIL-SCC_G,0.445276,HSIL-SCC_G,0.307003,HSIL_B,0.999537,41340,37756,230,264
9,NaN,NaN,NaN,341,NORMAL,0.555389,HSIL-SCC_G,0.107352,HSIL_B,0.999374,12972,10220,202,296


### features

In [6]:
tolerate = {"AGC":{"AGC_A", "AGC_B"}, 
            "LSIL":{"ASCUS", "LSIL_E", "LSIL_F"}, 
            "ASCUS":{"ASCUS", "LSIL_E", "LSIL_F"}, 
            "HSIL-SCC_G":{"HSIL_B", "HSIL_M", "HSIL_S", "SCC_G"}, 
            "SCC_R":{"SCC_R"}, 
            "EC":{"EC"}, 
            "CC":{"CC"}, 
            "VIRUS":{"VIRUS"}, 
            "FUNGI":{"FUNGI"}, 
            "ACTINO":{"ACTINO"}, 
            "TRI":{"TRI"}, 
            "PH":{"PH"}, 
            "SC":{"SC", "RC", "MC", "GEC"}}

dtct_p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
clas_p = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99, 0.999]
header = ["{}_{:.2f}_{:.3f}".format(key, dp, cp) for key in tolerate for dp in dtct_p for cp in clas_p]
header.sort()
header_map = {key:i for i,key in enumerate(header)}

all_labels = {'ACTINO':0, 'AGC':1, 'ASCH':2, 'ASCUS':3, 'CC':4, 'EC':5, 'FUNGI':6, 'HSIL':7, 'HSV':8, 'LSIL':9, 'NILM':10, 'SCC':11, 'TRI':12, 'VIRUS':8}
bin_labels = {}
for l in all_labels:
    if l == 'NILM':
        bin_labels[l] = 0
    else:
        bin_labels[l] = 1
print(all_labels)
print(bin_labels)

{'EC': 5, 'CC': 4, 'ASCH': 2, 'VIRUS': 8, 'ASCUS': 3, 'HSIL': 7, 'HSV': 8, 'AGC': 1, 'FUNGI': 6, 'SCC': 11, 'LSIL': 9, 'ACTINO': 0, 'TRI': 12, 'NILM': 10}
{'EC': 1, 'CC': 1, 'ASCH': 1, 'VIRUS': 1, 'ASCUS': 1, 'HSIL': 1, 'HSV': 1, 'NILM': 0, 'AGC': 1, 'FUNGI': 1, 'LSIL': 1, 'ACTINO': 1, 'TRI': 1, 'SCC': 1}


In [39]:
def extract(f):
    df = pd.read_csv(f)
    features = [0 for i in range(len(header))]
    # check if is empty
    if df.detect_label.isnull().values.any():  
        return features * 2
    # cross levelup features
    for i,row in df.iterrows():
        for dp in dtct_p:
            for cp in clas_p:
                if row['classify_label'] in tolerate[row['detect_label']]:
                    if row['detect_probability'] > dp and row['classify_probability'] > cp:
                        key = "{}_{:.2f}_{:.3f}".format(row['detect_label'], dp, cp)
                        features[header_map[key]] += 1
    # patches balanced numbers
    try:
        patches = float(df.patches[df.patches.notnull()])
        features_pb = [f*2000/patches for f in features]
        features += features_pb
    except:
        print(f)
        features *= 2
    return features

def collect(data):
    X = []
    ya = []  # all labels
    yb = []  # binary labels
    names = []
    for f in data:
        features = extract(f)
        basename = os.path.splitext(os.path.basename(f))[0]
        ls = label[basename].split('+')
        if sum(features) == 0:
            continue
        for l in ls:
            a = all_labels[l]
            b = bin_labels[l]
            X.append(features)
            ya.append(a)
            yb.append(b)
            names.append(f)
    return X, ya, yb, names

def worker():
    files = data
    random.shuffle(files)
    random.shuffle(files)
    print("# files:", len(files))

    X, ya, yb, names = [], [], [], []
    
    executor = ProcessPoolExecutor(max_workers=36)
    tasks = []

    batch_size = 50
    for i in range(0, len(files), batch_size):
        batch = files[i : i+batch_size]
#         collect(batch)
        tasks.append(executor.submit(collect, batch))

    job_count = len(tasks)
    for future in as_completed(tasks):
        X_, ya_, yb_, names_ = future.result()  # get the returning result from calling fuction
        X += X_
        ya += ya_
        yb += yb_
        names += names_
        job_count -= 1
        if job_count % 8 == 0: 
            print("One Job Done, Remaining Job Count: %s" % (job_count))

    X = np.asarray(X)
    ya = np.asarray(ya)
    yb = np.asarray(yb)
    print(X.shape, ya.shape, yb.shape)
    
    return X, ya, yb, names

X, ya, yb, names = worker()

# files: 2803
./gnet2/TC19010123.csv
./gnet2/TC19005061.csv
./gnet2/TC17028922.csv
./gnet2/TC19010718.csv
./gnet2/TC19010041.csv
./gnet2/TC18029996.csv
./gnet2/TC16021707.csv
./gnet2/2018-03-15-12_15_58.csv
./gnet2/TC18052517.csv
./gnet2/TC18050230.csv
./gnet2/TC16059362.csv
./gnet2/TC19005276.csv
./gnet2/TC19005131.csv
./gnet2/2018-03-15-14_09_03.csv
./gnet2/TC19011495.csv
./gnet2/TC16053743.csv
./gnet2/TC17067520.csv
./gnet2/TC18050286.csv
./gnet2/TC19008174.csv
./gnet2/TC17067137.csv
./gnet2/TC19005112.csv
./gnet2/2018-01-15-15_32_42.csv
./gnet2/TC16005407.csv
./gnet2/TC17039708.csv
./gnet2/TC18015696.csv
./gnet2/TC19008236.csv
./gnet2/TC19011006.csv
./gnet2/TC19008232.csv
./gnet2/TC18055591.csv
./gnet2/TC17001535.csv
./gnet2/SZH1801619.csv
./gnet2/TC18056498.csv
./gnet2/TC17034673.csv
./gnet2/TC16041594.csv
./gnet2/TC18016664.csv
./gnet2/NJ18088483.csv
./gnet2/TC19008268.csv
./gnet2/TC19010783.csv
./gnet2/TC16001554.csv
./gnet2/2017-11-24-13_16_54.csv
./gnet2/TC19005268.csv
./gnet2

### classification

In [42]:
def split(X, y, mode, test_size, seed):
    random.seed(seed)
    N = 2 if mode == "bin" else 13
    idx = {i:[] for i in range(N)}
    for i,c in enumerate(y):
        idx[c].append(i)
    idx_t, idx_v = [], []
    for c,indices in idx.items():
        n = len(indices)
        idx_t += indices[:-int(n*test_size)]
        idx_v += indices[-int(n*test_size):]
    X_train = X[idx_t]
    X_valid = X[idx_v]
    y_train = y[idx_t]
    y_valid = y[idx_v]
    return X_train, X_valid, y_train, y_valid


def evaluate(y_valid, y_pred, mode):
    if mode == "bin":
        cur_labels = {0:'NILM', 1:'ABN'}
    else:
        cur_labels = {value:key for key,value in all_labels.items()}
    
    results = {i:[0,0,0] for i in range(len(cur_labels))}  # tp, fn, fp
    for t,p in zip(y_valid, y_pred):
        if t == p:
            results[t][0] += 1
        else:
            results[t][1] += 1
            results[p][2] += 1
    for i in range(len(results)):
        labeli = cur_labels[i]
        recall = results[i][0] / (results[i][0] + results[i][1]) if results[i][0] + results[i][1] != 0 else 0.0
        precision = results[i][0] / (results[i][0] + results[i][2]) if results[i][0] + results[i][2] != 0 else 0.0
        print(labeli, results[i][0] + results[i][1], ' recall = {:.4f}'.format(recall), 'precision = {:.4f}'.format(precision))
        
    
def classify(mode="bin"):  # mode = "bin" or "all"
    seed = 2018
    test_size = 0.2

    y = yb if mode == "bin" else ya
#     X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=seed)
    X_train, X_valid, y_train, y_valid = split(X, y, mode, test_size, seed)

    model = XGBClassifier(max_depth=15, 
                          n_jobs=24, 
                          subsample=0.8, 
                          colsample_bylevel=1,
                          colsample_bytree=0.6, 
                          scale_pos_weight=1, 
                          n_estimators=500, 
                          min_child_weight=1, 
                          learning_rate=0.1, 
                          gamma=0,
                          random_state=seed)
    eval_set = [(X_train, y_train), (X_valid, y_valid)]
    if mode == "bin":
        eval_metric = ["auc", "error"]
    else:
        eval_metric = ["merror"]
    model.fit(X_train, y_train, 
              early_stopping_rounds=50, 
              eval_metric=eval_metric, 
              eval_set=eval_set, 
              verbose=False)

    y_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    print("accuracy: {:.4f}".format(accuracy))
    evaluate(y_valid, y_pred, mode)

print('binary classification')
classify("bin")
print()
print('multilabel classification')
classify("all")

binary classification
accuracy: 0.9149
NILM 197  recall = 0.8528 precision = 0.8936
ABN 379  recall = 0.9472 precision = 0.9253

multilabel classification
accuracy: 0.7605
ACTINO 8  recall = 1.0000 precision = 0.7273
AGC 6  recall = 1.0000 precision = 0.8571
ASCH 24  recall = 0.0417 precision = 0.2500
ASCUS 84  recall = 0.6310 precision = 0.6023
CC 35  recall = 0.8000 precision = 0.8485
EC 7  recall = 0.7143 precision = 0.8333
FUNGI 61  recall = 0.7705 precision = 0.8103
HSIL 47  recall = 0.7872 precision = 0.6981
HSV 2  recall = 1.0000 precision = 0.5000
LSIL 71  recall = 0.7324 precision = 0.8000
NILM 197  recall = 0.8934 precision = 0.8111
SCC 14  recall = 0.7143 precision = 0.7692
TRI 16  recall = 0.6250 precision = 0.7692
